<a href="https://www.kaggle.com/code/javlonbekshukurov/survival-prediction-for-cirrhosis-patients?scriptVersionId=236110829" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline 
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import log_loss
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multiclassificationtask/sample_submission.csv
/kaggle/input/multiclassificationtask/train.csv
/kaggle/input/multiclassificationtask/test.csv


In [2]:
train=pd.read_csv('/kaggle/input/multiclassificationtask/train.csv')
train.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,2178.0,D-penicillamine,16374.0,F,N,N,N,N,0.5,263.0,3.20,43.0,1110.0,106.95,67.0,430.0,9.6,3.0,C
1,1,2644.0,D-penicillamine,17774.0,F,N,N,N,N,0.8,280.0,3.60,22.0,678.0,62.00,80.0,427.0,13.0,3.0,C
2,2,3069.0,Placebo,17844.0,F,N,N,N,N,1.1,408.0,4.40,54.0,2108.0,142.60,137.0,203.0,10.6,3.0,C
3,3,2216.0,Placebo,19221.0,F,N,Y,Y,N,0.8,252.0,3.70,36.0,843.0,55.80,56.0,336.0,9.6,4.0,C
4,4,2256.0,Placebo,21600.0,F,N,N,N,N,4.7,348.0,3.06,464.0,961.0,120.90,146.0,298.0,11.0,2.0,D


In [3]:
test=pd.read_csv('/kaggle/input/multiclassificationtask/test.csv')
test.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,15000,22646.0,NaN,22646.0,F,NaN,NaN,NaN,N,1.4,NaN,3.34,NaN,NaN,NaN,NaN,190.0,11.0,4.0
1,15001,2149.0,NaN,18993.0,F,NaN,NaN,NaN,N,0.9,NaN,3.24,NaN,NaN,NaN,NaN,233.0,9.0,2.0
2,15002,3850.0,D-penicillamine,16050.0,M,N,N,N,N,1.0,460.0,3.64,75.0,2148.0,102.3,118.0,172.0,10.6,3.0
3,15003,2419.0,D-penicillamine,20354.0,F,N,N,N,N,0.6,258.0,3.96,31.0,642.0,70.0,83.0,336.0,10.8,2.0
4,15004,1086.0,NaN,23376.0,F,NaN,NaN,NaN,N,4.4,NaN,4.40,NaN,NaN,NaN,NaN,447.0,10.3,2.0


## **Step 1 : Data Analysis**

In [4]:
train.shape

(15000, 20)

In [5]:
train.isnull().sum()

id                  0
N_Days              0
Drug             6506
Age                 0
Sex                 0
Ascites          6498
Hepatomegaly     6508
Spiders          6509
Edema               0
Bilirubin           0
Cholesterol      8299
Albumin             0
Copper           6601
Alk_Phos         6512
SGOT             6514
Tryglicerides    8334
Platelets         564
Prothrombin        16
Stage               0
Status              0
dtype: int64

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15000 non-null  int64  
 1   N_Days         15000 non-null  float64
 2   Drug           8494 non-null   object 
 3   Age            15000 non-null  float64
 4   Sex            15000 non-null  object 
 5   Ascites        8502 non-null   object 
 6   Hepatomegaly   8492 non-null   object 
 7   Spiders        8491 non-null   object 
 8   Edema          15000 non-null  object 
 9   Bilirubin      15000 non-null  float64
 10  Cholesterol    6701 non-null   float64
 11  Albumin        15000 non-null  float64
 12  Copper         8399 non-null   float64
 13  Alk_Phos       8488 non-null   float64
 14  SGOT           8486 non-null   float64
 15  Tryglicerides  6666 non-null   float64
 16  Platelets      14436 non-null  float64
 17  Prothrombin    14984 non-null  float64
 18  Stage 

In [7]:
train.Drug.value_counts()

Drug
D-penicillamine    4610
Placebo            3884
Name: count, dtype: int64

In [8]:
train.Ascites .value_counts()

Ascites
N    8116
Y     386
Name: count, dtype: int64

In [9]:
train.Hepatomegaly.value_counts()

Hepatomegaly
N    4634
Y    3858
Name: count, dtype: int64

In [10]:
train.Spiders .value_counts()

Spiders
N    6769
Y    1722
Name: count, dtype: int64

In [11]:
train.drop_duplicates(inplace=True)



**From the Caterogical columns, I planned to fill nan values such a way:**

**1. Columns which weighed to one side will be filled this most frequent value**

**2. Columns which have big difference between values will be filled with 'ffill' method**




**For Numerical columns, nan values will be filled with 'median' method**



In [12]:
train['age_in_years']=(train['Age']/365).astype(int)
train.drop('Age', axis=1, inplace=True)
train.head()

,id,N_Days,Drug,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status,age_in_years
0,0,2178.0,D-penicillamine,F,N,N,N,N,0.5,263.0,3.20,43.0,1110.0,106.95,67.0,430.0,9.6,3.0,C,44
1,1,2644.0,D-penicillamine,F,N,N,N,N,0.8,280.0,3.60,22.0,678.0,62.00,80.0,427.0,13.0,3.0,C,48
2,2,3069.0,Placebo,F,N,N,N,N,1.1,408.0,4.40,54.0,2108.0,142.60,137.0,203.0,10.6,3.0,C,48
3,3,2216.0,Placebo,F,N,Y,Y,N,0.8,252.0,3.70,36.0,843.0,55.80,56.0,336.0,9.6,4.0,C,52
4,4,2256.0,Placebo,F,N,N,N,N,4.7,348.0,3.06,464.0,961.0,120.90,146.0,298.0,11.0,2.0,D,59


In [13]:
train.head()

,id,N_Days,Drug,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status,age_in_years
0,0,2178.0,D-penicillamine,F,N,N,N,N,0.5,263.0,3.20,43.0,1110.0,106.95,67.0,430.0,9.6,3.0,C,44
1,1,2644.0,D-penicillamine,F,N,N,N,N,0.8,280.0,3.60,22.0,678.0,62.00,80.0,427.0,13.0,3.0,C,48
2,2,3069.0,Placebo,F,N,N,N,N,1.1,408.0,4.40,54.0,2108.0,142.60,137.0,203.0,10.6,3.0,C,48
3,3,2216.0,Placebo,F,N,Y,Y,N,0.8,252.0,3.70,36.0,843.0,55.80,56.0,336.0,9.6,4.0,C,52
4,4,2256.0,Placebo,F,N,N,N,N,4.7,348.0,3.06,464.0,961.0,120.90,146.0,298.0,11.0,2.0,D,59


In [14]:
train=train[train['Status'] != 'Y']
X=train.drop(columns=['id','Status'])
y=train['Status']

In [15]:
test['age_in_years']=(test['Age']/365).astype(int)
test.drop(['Age','id'], axis=1, inplace=True)
test.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,N_Days,Drug,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,age_in_years
0,22646.0,NaN,F,NaN,NaN,NaN,N,1.4,NaN,3.34,NaN,NaN,NaN,NaN,190.0,11.0,4.0,62
1,2149.0,NaN,F,NaN,NaN,NaN,N,0.9,NaN,3.24,NaN,NaN,NaN,NaN,233.0,9.0,2.0,52
2,3850.0,D-penicillamine,M,N,N,N,N,1.0,460.0,3.64,75.0,2148.0,102.3,118.0,172.0,10.6,3.0,43
3,2419.0,D-penicillamine,F,N,N,N,N,0.6,258.0,3.96,31.0,642.0,70.0,83.0,336.0,10.8,2.0,55
4,1086.0,NaN,F,NaN,NaN,NaN,N,4.4,NaN,4.40,NaN,NaN,NaN,NaN,447.0,10.3,2.0,64


In [16]:
test.isnull().sum()

N_Days              0
Drug             4398
Sex                 0
Ascites          4391
Hepatomegaly     4402
Spiders          4404
Edema               0
Bilirubin           0
Cholesterol      5647
Albumin             0
Copper           4469
Alk_Phos         4403
SGOT             4406
Tryglicerides    5680
Platelets         382
Prothrombin         9
Stage               0
age_in_years        0
dtype: int64

## **Step 2 : Pipeline**

In [17]:
most_frequent_cols=['Ascites','Spiders','Sex','Edema']
ffill_cols = ['Drug', 'Hepatomegaly']
num_cols = X.select_dtypes(include=['number']).columns.tolist()

In [18]:
class ForwardFillImputer(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return pd.DataFrame(X).ffill().values

In [19]:
most_freq_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
ffill_pipeline = Pipeline([
    ('ffill', ForwardFillImputer()),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [20]:
full_pipeline = ColumnTransformer([
    ('most_freq_cat', most_freq_pipeline, most_frequent_cols),
    ('ffill_cat', ffill_pipeline, ffill_cols),
    ('num', num_pipeline, num_cols)
])

In [21]:
X_transformed=full_pipeline.fit_transform(X)
X_transformed[0]

array([ 1.        ,  0.        ,  1.        ,  0.        ,  1.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  1.        ,  0.        ,  0.15478384, -0.50302697,
       -0.31691144, -0.87625475, -0.38883121, -0.19997958,  0.09943872,
       -1.00801519,  1.92120314, -1.40389741, -0.02838968, -0.81986736])

## **Step 3: Train and Test set**

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

## **Step 4 : Machine Learning Models**

## **LogisticRegression Model**

In [23]:
LR_model=LogisticRegression(multi_class='multinomial', solver='lbfgs')
LR_model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

In [24]:
# Model evaluation
y_pred_proba = LR_model.predict_proba(X_test)
eps = 1e-15
y_pred_clipped = np.clip(y_pred_proba, eps, 1 - eps)
loss = log_loss(y_test, y_pred_clipped)
print("Log Loss: " ,loss)

Log Loss:  0.4654958960008868


## **RandomForestClassifier Model**

In [25]:
RF_model=RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
RF_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, random_state=42)

In [26]:
y_pred_proba = RF_model.predict_proba(X_test)
eps = 1e-15
y_pred_clipped = np.clip(y_pred_proba, eps, 1 - eps)
loss = log_loss(y_test, y_pred_clipped)
print("Log Loss: " ,loss)

Log Loss:  0.4376148396242809


## **GradientBoostingClassifier Model**

In [27]:
GB_model=GradientBoostingClassifier(n_estimators=1000, learning_rate=0.05, max_depth=5, random_state=42)
GB_model.fit(X_train, y_train)
y_preda_proba = GB_model.predict_proba(X_test)
eps = 1e-15
y_pred_clipped = np.clip(y_preda_proba, eps, 1 - eps)

In [28]:
loss = log_loss(y_test, y_pred_clipped)
print("Log Loss: " ,loss)

Log Loss:  0.41633220630297835


## **XGBoost Classifier Model**

In [29]:
le=LabelEncoder()
y_train_encoded=le.fit_transform(y_train)
y_test_encoded=le.fit_transform(y_test)

In [30]:
XGB_model = xgb.XGBClassifier(use_label_encoder=False,learning_rate=0.11,max_depth=5,n_estimators=180,eval_metric='logloss',random_state=42,subsample=0.7,colsample_bytree=0.2)
XGB_model.fit(X_train, y_train_encoded)
y_pred=XGB_model.predict_proba(X_test)
eps = 1e-15
y_pred_clipped = np.clip(y_pred, eps, 1 - eps)

In [31]:
loss = log_loss(y_test_encoded, y_pred_clipped)
print("Log Loss: " ,loss)

Log Loss:  0.3657992516063562


**The least loss** was reached by **XGBoost Classifier** with log_loss : **0.36**57.....

## **Step 6 : Submission**

In [32]:
test_transformed=full_pipeline.transform(test)

In [33]:
test_predictions = XGB_model.predict_proba(test_transformed)

if 'id' in test.columns:
    submission = pd.DataFrame({'id': test['id']})
else:
    submission = pd.DataFrame({'id': range(15000, 15000+len(test))})

submission[['Status_C', 'Status_CL', 'Status_D']] = test_predictions

submission.to_csv('submission.csv', index=False)
print(submission)

         id  Status_C  Status_CL  Status_D
0     15000  0.396621   0.006098  0.597281
1     15001  0.886430   0.013442  0.100128
2     15002  0.556348   0.040293  0.403359
3     15003  0.991425   0.000520  0.008054
4     15004  0.333008   0.192984  0.474008
...     ...       ...        ...       ...
9995  24995  0.922124   0.018099  0.059778
9996  24996  0.038570   0.007853  0.953577
9997  24997  0.900750   0.011177  0.088072
9998  24998  0.951165   0.003278  0.045557
9999  24999  0.595088   0.053178  0.351734

[10000 rows x 4 columns]
